In [13]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')

In [23]:
Flow_Freq<-read.csv("Flow_Frequency.csv")
scRNA_Freq<-read.csv("freq_actual_flu_year_1_D0_D7.csv")

In [24]:
Flow_Freq <- Flow_Freq %>%
  dplyr::mutate(AIFI_L3_matched = case_when(
    celltype == 'CD27_neg_Effector' ~ 'CD27- effector B cell',
    celltype == 'CD27_pos_Effector' ~ 'CD27+ effector B cell',
    celltype == 'Early_Memory' ~ 'Early memory B cell',
    celltype == 'Core_Memory' ~ 'Core memory B cell',
    celltype == 'CD95_Memory' ~ 'CD95 memory B cell',
    celltype == 'Transitional' ~ 'Transitional B cell',
    celltype == 'Naive' ~ 'Core naive B cell',
    celltype == 'Plasma' ~ 'Plasma cell',
    celltype == 'T2MBC' ~ 'Type 2 polarized memory B cell',
    TRUE ~ as.character(celltype)
  )) %>% filter(!timepoint=='day90')

In [25]:
Flow_Freq <- Flow_Freq %>%
  dplyr::mutate(sample.visitName = case_when(
    timepoint == 'day0' ~ 'Flu Year 1 Day 0',
    timepoint == 'day7' ~ 'Flu Year 1 Day 7',
    TRUE ~ as.character(timepoint)
  )) 

In [26]:
scRNA_Freq<-scRNA_Freq %>%
  dplyr::mutate(AIFI_L3_matched = case_when(
    AIFI_L3 %in% c( unique(scRNA_Freq$AIFI_L3)[grepl('B cell',unique(scRNA_Freq$AIFI_L3))],'Plasma cell') ~ as.character(AIFI_L3),
    TRUE ~ 'non_B'
  )) %>%
  dplyr::mutate(AIFI_L3_matched = case_when(
    AIFI_L3_matched %in% c( 'Activated memory B cell') ~ 'Core memory B cell',
    AIFI_L3_matched %in% c( 'ISG+ naive B cell') ~ 'Core naive B cell',  
    TRUE ~ as.character(AIFI_L3_matched)
  ))

In [27]:
scRNA_Freq_agg <- scRNA_Freq %>%
  group_by(subject.subjectGuid, sample.visitName,AIFI_L3_matched) %>%
  summarize(Counts = sum(AIFI_L3_count), .groups = 'drop')

In [28]:
scRNA_Freq_agg<-scRNA_Freq_agg %>% group_by(subject.subjectGuid,sample.visitName) %>% mutate(frequency=Counts/sum(Counts)*100) %>% mutate(frequency_clr=clr_transform(frequency))

In [29]:
colnames(scRNA_Freq_agg)<-c('subject.subjectGuid','sample.visitName','AIFI_L3_matched','Counts_scRNA','frequency_scRNA','frequency_clr_scRNA')
colnames(Flow_Freq)<-c('row','subject.subjectGuid','timepoint','celltype','Counts_Flow','frequency_Flow','frequency_clr_Flow','AIFI_L3_matched','sample.visitName')

In [32]:
df_combined<-left_join(Flow_Freq,scRNA_Freq_agg)

Joining with `by = join_by(subject.subjectGuid, AIFI_L3_matched,
sample.visitName)`


In [34]:
write.csv(df_combined,'flow_scRNA_Freq.csv')